## 表1

In [1]:
import pandas as pd
import statsmodels.api as sm
from linearmodels.panel import PanelOLS, RandomEffects
import numpy as np

# ---------------------------------------------------
# 1. Data Preparation
# ---------------------------------------------------

# 读取数据
file_path = r'2002-2017 Panel Data.xls'
data = pd.read_excel(file_path, sheet_name='Mid-tier IND')

# 重命名列，避免使用特殊字符
data.rename(columns={'GFCF%GDP': 'GFCF_GDP'}, inplace=True)

# 选择相关列并删除缺失值
required_columns = [
    'emp_lg', 'rd_exp', 'gdp_g', 'inf', 'pop_g', 
    'GFCF_GDP', 'rd_receive', 'rd_self', 'Industry', 'Year'
]
data = data[required_columns].dropna()
data = data.replace("-", np.nan)

# 转换数据类型
data[['rd_receive', 'rd_self']] = data[['rd_receive', 'rd_self']].apply(pd.to_numeric)

# 设置多层索引（面板数据格式：行业和年份）
data = data.set_index(['Industry', 'Year'])

# ---------------------------------------------------
# 2. Define Utility Functions
# ---------------------------------------------------

def run_model(model_type, formula, data, re_formula=None):
    """
    Runs the specified model type with the given formula and data.
    """
    if model_type == 'pooled':
        X = sm.add_constant(data[formula['exog']])
        y = data[formula['endog']]
        model = sm.OLS(y, X).fit()
        return model
    elif model_type == 'fe':
        model = PanelOLS.from_formula(formula['formula'], data).fit()
        return model
    elif model_type == 're':
        model = RandomEffects.from_formula(formula['formula'], data).fit()
        return model
    else:
        raise ValueError("Unsupported model type.")

def format_pvalue(pval):
    """
    将 p 值格式化为显著性星号。
    """
    if pval < 0.01:
        return "***"
    elif pval < 0.05:
        return "**"
    elif pval < 0.1:
        return "*"
    else:
        return " "

def extract_coefficients(model, model_type):
    """
    Extracts coefficients and p-values from the model and formats them with significance stars.
    """
    coeffs = {}
    
    if model_type == 'fe':
        # For Fixed Effects, use the mean of entity fixed effects as the constant
        constant = model.estimated_effects.mean().values[0]
        coeffs['const'] = f"{constant:.3f}"  # No p-value for FE constant
    else:
        # For other models, attempt to extract 'const' or 'Intercept'
        if 'const' in model.params.index:
            coeff = f"{model.params['const']:.3f}"
            star = format_pvalue(model.pvalues['const'])
            coeffs['const'] = f"{coeff}{star}"
        elif 'Intercept' in model.params.index:
            coeff = f"{model.params['Intercept']:.3f}"
            star = format_pvalue(model.pvalues['Intercept'])
            coeffs['const'] = f"{coeff}{star}"
        else:
            coeffs['const'] = ''

    # Extract other coefficients
    for var in ['rd_exp', 'rd_receive', 'rd_self', 'gdp_g', 'inf', 'pop_g', 'GFCF_GDP']:
        if var in model.params.index:
            coeff = f"{model.params[var]:.3f}"
            pval = model.pvalues[var]
            star = format_pvalue(pval)
            coeffs[var] = f"{coeff}{star}"
        else:
            coeffs[var] = ''

    # 根据模型类型选择 R_squared
    r_squared = model.rsquared
    if model_type == 're':
        r_squared = model.rsquared_between
    if model_type == 'fe':
        r_squared = model.rsquared_overall

    #rsquared_within
    coeffs['R_squared'] = round(r_squared, 2) if r_squared is not None else None
    return coeffs

def prepare_results_df(models):
    """
    Prepares a consolidated DataFrame of all model results.
    """
    variables = ['const', 'rd_exp', 'rd_receive', 'rd_self', 'gdp_g', 'inf', 'pop_g', 'GFCF_GDP']
    results = pd.DataFrame(index=variables + ['R_squared'])

    for name, model_info in models.items():
        coeffs = extract_coefficients(model_info['model'], model_info['type'])
        results[name] = [coeffs.get(var) for var in variables] + [coeffs.get('R_squared')]

    # Rename 'const' to 'Constant' for clarity
    results.rename(index={'const': 'Constant'}, inplace=True)

    return results

# ---------------------------------------------------
# 3. Run Models
# ---------------------------------------------------

# Define formulas for each model
formulas = {
    'pooled': {
        'type': 'pooled',
        'exog': ['rd_exp', 'gdp_g', 'inf', 'pop_g', 'GFCF_GDP'],
        'endog': 'emp_lg'
    },
    'fe': {
        'type': 'fe',
        'formula': 'emp_lg ~ rd_exp + gdp_g + inf + pop_g + GFCF_GDP + EntityEffects'
    },
    're': {
        'type': 're',
        'formula': 'emp_lg ~ rd_exp + gdp_g + inf + pop_g + GFCF_GDP + 1'
    },
    're_rd_receive': {
        'type': 're',
        'formula': 'emp_lg ~ rd_receive + gdp_g + inf + pop_g + GFCF_GDP + 1'
    },
    're_rd_self': {
        'type': 're',
        'formula': 'emp_lg ~ rd_self + gdp_g + inf + pop_g + GFCF_GDP + 1'
    }
}

# Run all models and store them in a dictionary
models = {}
for name, formula in formulas.items():
    models[name] = {
        'model': run_model(formula['type'], formula, data),
        'type': formula['type']
    }

# ---------------------------------------------------
# 4. Consolidate Results
# ---------------------------------------------------

results_df = prepare_results_df(models)

# ---------------------------------------------------
# 5. Display Results
# ---------------------------------------------------
results_df


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/linearmodels/panel/model.py:2759: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/linearmodels/panel/model.py:2759: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


,pooled,fe,re,re_rd_receive,re_rd_self
Constant,-2.087*,-0.151,-0.390,3.260***,-0.397
rd_exp,0.495***,0.321***,0.343***,,
rd_receive,,,,0.023*,
rd_self,,,,,0.351***
gdp_g,0.004,0.002,0.003,-0.002,0.001
inf,-0.018,-0.018,-0.018,-0.019,-0.020
pop_g,0.167,0.252**,0.241**,0.251**,0.344***
GFCF_GDP,0.041,0.044***,0.044***,0.049***,0.040***
R_squared,0.6,0.97,0.56,0.04,0.63


In [3]:
# 修改行顺序，变成报告中表格的样式
results_df.iloc[[1,2,3,4,5,6,7,0,8],]

,pooled,fe,re,re_rd_receive,re_rd_self
rd_exp,0.495***,0.321***,0.343***,,
rd_receive,,,,0.023*,
rd_self,,,,,0.351***
gdp_g,0.004,0.002,0.003,-0.002,0.001
inf,-0.018,-0.018,-0.018,-0.019,-0.020
pop_g,0.167,0.252**,0.241**,0.251**,0.344***
GFCF_GDP,0.041,0.044***,0.044***,0.049***,0.040***
Constant,-2.087*,-0.151,-0.390,3.260***,-0.397
R_squared,0.6,0.97,0.56,0.04,0.63


# 表2

In [4]:
import pandas as pd
import statsmodels.api as sm
from linearmodels.panel import PanelOLS, RandomEffects
import numpy as np

# ---------------------------------------------------
# 1. 数据准备
# ---------------------------------------------------

# 读取数据
file_path = r'2002-2017 Panel Data.xls'
data = pd.read_excel(file_path, sheet_name='Mid-tier IND')

# 重命名列，避免使用特殊字符
data.rename(columns={'GFCF%GDP': 'GFCF_GDP'}, inplace=True)

# 选择相关列并删除缺失值，新增 'male' 和 'age'
required_columns = [
    'emp_lg', 'rd_exp', 'gdp_g', 'inf', 'pop_g', 
    'GFCF_GDP', 'rd_receive', 'rd_self', 'Industry', 'Year',
    'male', 'age'  # 新增的变量
]
data = data[required_columns].dropna()
data = data.replace("-", np.nan)

# 转换数据类型
data[['rd_receive', 'rd_self', 'male', 'age']] = data[['rd_receive', 'rd_self', 'male', 'age']].apply(pd.to_numeric)

# 设置多层索引（面板数据格式：行业和年份）
data = data.set_index(['Industry', 'Year'])

# ---------------------------------------------------
# 2. 定义辅助函数
# ---------------------------------------------------

def run_model(model_type, formula, data, re_formula=None):
    """
    运行指定类型的模型。
    """
    if model_type == 'pooled':
        X = sm.add_constant(data[formula['exog']])
        y = data[formula['endog']]
        model = sm.OLS(y, X).fit()
        return model
    elif model_type == 'fe':
        model = PanelOLS.from_formula(formula['formula'], data).fit()
        return model
    elif model_type == 're':
        model = RandomEffects.from_formula(formula['formula'], data).fit()
        return model
    else:
        raise ValueError("不支持的模型类型。")

def format_pvalue(pval):
    """
    将 p 值格式化为显著性星号。
    """
    if pval < 0.01:
        return "***"
    elif pval < 0.05:
        return "**"
    elif pval < 0.1:
        return "*"
    else:
        return " "

def extract_coefficients(model, model_type):
    """
    从模型中提取系数和 p 值，并格式化显示。
    """
    coeffs = {}
    
    if model_type == 'fe':
        # 对于固定效应模型，使用实体固定效应的均值作为常数
        constant = model.estimated_effects.mean().values[0]
        coeffs['const'] = f"{constant:.3f}"  # 固定效应模型的常数没有 p 值
    else:
        # 对于其他模型，尝试提取 'const' 或 'Intercept'
        if 'const' in model.params.index:
            coeff = f"{model.params['const']:.3f}"
            star = format_pvalue(model.pvalues['const'])
            coeffs['const'] = f"{coeff}{star}"
        elif 'Intercept' in model.params.index:
            coeff = f"{model.params['Intercept']:.3f}"
            star = format_pvalue(model.pvalues['Intercept'])
            coeffs['const'] = f"{coeff}{star}"
        else:
            coeffs['const'] = None

    # 提取其他系数
    # 新增的变量包括 'male', 'age', 'rd_exp:male'（交互项）
    variables = ['rd_exp', 'rd_receive', 'rd_self', 'gdp_g', 'inf', 'pop_g', 'GFCF_GDP', 'male', 'age', 'rd_exp:male']
    for var in variables:
        if var in model.params.index:
            coeff = f"{model.params[var]:.3f}"
            pval = model.pvalues[var]
            star = format_pvalue(pval)
            coeffs[var] = f"{coeff}{star}"
        elif var.replace(':', ' * ') in model.params.index:
            # 处理交互项的命名
            interaction_var = var.replace(':', ' * ')
            if interaction_var in model.params.index:
                coeff = f"{model.params[interaction_var]:.3f}"
                pval = model.pvalues[interaction_var]
                star = format_pvalue(pval)
                coeffs[var] = f"{coeff}{star}"
            else:
                coeffs[var] = ''
        else:
            coeffs[var] = ''

    r_squared = model.rsquared
    if model_type == 're':
        r_squared = model.rsquared_between
    if model_type == 'fe':
        r_squared = model.rsquared_overall
    coeffs['R_squared'] = round(r_squared, 2) if r_squared is not None else None

    return coeffs

def prepare_results_df(models):
    """
    整理所有模型的结果到一个 DataFrame 中。
    """
    # 更新变量列表，包含新增变量
    variables = ['const', 'rd_exp', 'rd_receive', 'rd_self', 'gdp_g', 'inf', 'pop_g', 'GFCF_GDP', 'male', 'age', 'rd_exp:male']
    results = pd.DataFrame(index=variables + ['R_squared'])

    for name, model_info in models.items():
        coeffs = extract_coefficients(model_info['model'], model_info['type'])
        results[name] = [coeffs.get(var) for var in variables] + [coeffs.get('R_squared')]

    # 将 'const' 重命名为 'Constant' 以便清晰显示
    results.rename(index={'const': 'Constant'}, inplace=True)

    return results

# ---------------------------------------------------
# 3. 运行模型
# ---------------------------------------------------

# 定义每个模型的公式，所有模型均添加 'male' 和 'age'
formulas = {
    'pooled': {
        'type': 'pooled',
        'exog': ['rd_exp', 'gdp_g', 'inf', 'pop_g', 'GFCF_GDP', 'male', 'age'],
        'endog': 'emp_lg'
    },
    'fe': {
        'type': 'fe',
        'formula': 'emp_lg ~ rd_exp + gdp_g + inf + pop_g + GFCF_GDP + male + age + EntityEffects'
    },
    're': {
        'type': 're',
        'formula': 'emp_lg ~ rd_exp + gdp_g + inf + pop_g + GFCF_GDP + male + age + 1'
    },
    're_rd_receive': {
        'type': 're',
        'formula': 'emp_lg ~ rd_receive + gdp_g + inf + pop_g + GFCF_GDP + male + age + 1'
    },
    're_rd_self': {
        'type': 're',
        'formula': 'emp_lg ~ rd_self + gdp_g + inf + pop_g + GFCF_GDP + male + age + 1'
    },
    # 新增第六个模型：RE模型 + male + age + rd_exp * male
    're_interaction': {
        'type': 're',
        'formula': 'emp_lg ~ rd_exp + gdp_g + inf + pop_g + GFCF_GDP + male + age + rd_exp:male + 1'
    }
}

# 运行所有模型并存储在字典中
models = {}
for name, formula in formulas.items():
    models[name] = {
        'model': run_model(formula['type'], formula, data),
        'type': formula['type']
    }

# ---------------------------------------------------
# 4. 整理结果
# ---------------------------------------------------

results_df = prepare_results_df(models)

# ---------------------------------------------------
# 5. 显示结果
# ---------------------------------------------------
results_df

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/linearmodels/panel/model.py:2759: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/linearmodels/panel/model.py:2759: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


,pooled,fe,re,re_rd_receive,re_rd_self,re_interaction
Constant,-1.777,-0.114,-0.322,2.950***,-0.092,-1.893
rd_exp,0.497***,0.321***,0.341***,,,0.521***
rd_receive,,,,0.025*,,
rd_self,,,,,0.350***,
gdp_g,0.004,0.001,0.002,-0.003,0.001,0.001
inf,-0.017,-0.019,-0.019,-0.020,-0.019,-0.018
pop_g,0.186,0.325*,0.309*,0.360**,0.335**,0.283*
GFCF_GDP,0.037,0.040***,0.039***,0.046***,0.037***,0.036**
male,-0.005,-0.004,-0.004,-0.003,-0.003,0.017
age,0.003,0.008,0.007,0.011,0.000,0.007


In [5]:
# 修改行顺序，变成报告中表格的样式
results_df.iloc[[1,2,3,4,5,6,7,8,9,10,0,11],]

,pooled,fe,re,re_rd_receive,re_rd_self,re_interaction
rd_exp,0.497***,0.321***,0.341***,,,0.521***
rd_receive,,,,0.025*,,
rd_self,,,,,0.350***,
gdp_g,0.004,0.001,0.002,-0.003,0.001,0.001
inf,-0.017,-0.019,-0.019,-0.020,-0.019,-0.018
pop_g,0.186,0.325*,0.309*,0.360**,0.335**,0.283*
GFCF_GDP,0.037,0.040***,0.039***,0.046***,0.037***,0.036**
male,-0.005,-0.004,-0.004,-0.003,-0.003,0.017
age,0.003,0.008,0.007,0.011,0.000,0.007
rd_exp:male,,,,,,-0.002


# 表3

In [6]:
import pandas as pd
import statsmodels.api as sm
from linearmodels.panel import PanelOLS, RandomEffects
import numpy as np

# ---------------------------------------------------
# 1. 数据准备
# ---------------------------------------------------

# 读取数据
file_path = r'2002-2017 Panel Data.xls'
data = pd.read_excel(file_path, sheet_name='Manufacture')

# 重命名列，避免使用特殊字符
data.rename(columns={'GFCF%GDP': 'GFCF_GDP'}, inplace=True)

# 选择相关列并删除缺失值，新增 'male' 和 'age'
required_columns = [
    'emp', 'rd_exp', 'gdp_g', 'inf', 'pop_g', 
    'GFCF_GDP', 'rd_receive', 'rd_self', 'Industry', 'Year',
    'male', 'age'  # 新增的变量
]
data = data[required_columns].dropna()
data = data.replace("-", np.nan)
data.loc[:,'agesq']=data.loc[:,'age']**2

# 转换数据类型
data[['rd_receive', 'rd_self', 'male', 'age']] = data[['rd_receive', 'rd_self', 'male', 'age']].apply(pd.to_numeric)

# 设置多层索引（面板数据格式：行业和年份）
data = data.set_index(['Industry', 'Year'])

# ---------------------------------------------------
# 2. 定义辅助函数
# ---------------------------------------------------

def run_model(model_type, formula, data, re_formula=None):
    """
    运行指定类型的模型。
    """
    if model_type == 'pooled':
        X = sm.add_constant(data[formula['exog']])
        y = data[formula['endog']]
        model = sm.OLS(y, X).fit()
        return model
    elif model_type == 'fe':
        model = PanelOLS.from_formula(formula['formula'], data).fit()
        return model
    elif model_type == 're':
        model = RandomEffects.from_formula(formula['formula'], data).fit()
        return model
    else:
        raise ValueError("不支持的模型类型。")

def format_pvalue(pval):
    """
    将 p 值格式化为显著性星号。
    """
    if pval < 0.01:
        return "***"
    elif pval < 0.05:
        return "**"
    elif pval < 0.1:
        return "*"
    else:
        return " "

def extract_coefficients(model, model_type):
    """
    从模型中提取系数和 p 值，并格式化显示。
    """
    coeffs = {}
    
    if model_type == 'fe':
        # 对于固定效应模型，使用实体固定效应的均值作为常数
        constant = model.estimated_effects.mean().values[0]
        coeffs['const'] = f"{constant:.3f}"  # 固定效应模型的常数没有 p 值
    else:
        # 对于其他模型，尝试提取 'const' 或 'Intercept'
        if 'const' in model.params.index:
            coeff = f"{model.params['const']:.3f}"
            star = format_pvalue(model.pvalues['const'])
            coeffs['const'] = f"{coeff}{star}"
        elif 'Intercept' in model.params.index:
            coeff = f"{model.params['Intercept']:.3f}"
            star = format_pvalue(model.pvalues['Intercept'])
            coeffs['const'] = f"{coeff}{star}"
        else:
            coeffs['const'] = None

    # 提取其他系数
    # 新增的变量包括 'male', 'age', 'rd_exp:male'（交互项）
    variables = ['rd_exp', 'rd_receive', 'rd_self', 'gdp_g', 'inf', 'pop_g', 'GFCF_GDP', 'male', 'age', 'agesq']
    for var in variables:
        if var in model.params.index:
            coeff = f"{model.params[var]:.3f}"
            pval = model.pvalues[var]
            star = format_pvalue(pval)
            coeffs[var] = f"{coeff}{star}"
        elif var.replace(':', ' * ') in model.params.index:
            # 处理交互项的命名
            interaction_var = var.replace(':', ' * ')
            if interaction_var in model.params.index:
                coeff = f"{model.params[interaction_var]:.3f}"
                pval = model.pvalues[interaction_var]
                star = format_pvalue(pval)
                coeffs[var] = f"{coeff}{star}"
            else:
                coeffs[var] = ''
        else:
            coeffs[var] = ''

    r_squared = model.rsquared
    if model_type == 're':
        r_squared = model.rsquared_between
    if model_type == 'fe':
        r_squared = model.rsquared_overall
    coeffs['R_squared'] = round(r_squared, 2) if r_squared is not None else None

    return coeffs

def prepare_results_df(models):
    """
    整理所有模型的结果到一个 DataFrame 中。
    """
    # 更新变量列表，包含新增变量
    variables = ['const', 'rd_exp', 'rd_receive', 'rd_self', 'gdp_g', 'inf', 'pop_g', 'GFCF_GDP', 'male', 'age', 'agesq']
    results = pd.DataFrame(index=variables + ['R_squared'])

    for name, model_info in models.items():
        coeffs = extract_coefficients(model_info['model'], model_info['type'])
        results[name] = [coeffs.get(var) for var in variables] + [coeffs.get('R_squared')]

    # 将 'const' 重命名为 'Constant' 以便清晰显示
    results.rename(index={'const': 'Constant'}, inplace=True)

    return results

# ---------------------------------------------------
# 3. 运行模型
# ---------------------------------------------------

# 定义每个模型的公式，所有模型均添加 'male' 和 'age'
formulas = {
    're1': {
        'type': 're',
        'formula': 'emp ~ rd_exp + gdp_g + inf + pop_g + GFCF_GDP + 1'        
    },
    're2': {
        'type': 're',
        'formula': 'emp ~ rd_receive + gdp_g + inf + pop_g + GFCF_GDP + 1'
    },
    're3': {
        'type': 're',
        'formula': 'emp ~ rd_self + gdp_g + inf + pop_g + GFCF_GDP + 1'
    },
    're4': {
        'type': 're',
        'formula': 'emp ~ rd_exp + gdp_g + inf + pop_g + GFCF_GDP + male + age + agesq + 1'
    },
    're5': {
        'type': 're',
        'formula': 'emp ~ rd_receive + gdp_g + inf + pop_g + GFCF_GDP + male + age + agesq + 1'
    },
    're6': {
        'type': 're',
        'formula': 'emp ~ rd_self + gdp_g + inf + pop_g + GFCF_GDP + male + age + agesq + 1'
    }
}

# 运行所有模型并存储在字典中
models = {}
for name, formula in formulas.items():
    models[name] = {
        'model': run_model(formula['type'], formula, data),
        'type': formula['type']
    }

# ---------------------------------------------------
# 4. 整理结果
# ---------------------------------------------------

results_df = prepare_results_df(models)

# ---------------------------------------------------
# 5. 显示结果
# ---------------------------------------------------
results_df

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/linearmodels/panel/model.py:2759: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/linearmodels/panel/model.py:2759: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


,re1,re2,re3,re4,re5,re6
Constant,-0.399,3.413***,-0.402,-37.694*,-13.671,-39.112*
rd_exp,0.388***,,,0.389***,,
rd_receive,,0.101***,,,0.101***,
rd_self,,,0.387***,,,0.390***
gdp_g,0.000,-0.004,0.000,-0.006,-0.009,-0.006
inf,-0.016,-0.015,-0.017,-0.018,-0.016,-0.018
pop_g,0.295***,0.254**,0.315***,0.294*,0.352**,0.286
GFCF_GDP,0.026**,0.032**,0.025**,0.002,0.015,0.002
male,,,,-0.064**,-0.035,-0.066**
age,,,,1.675*,0.779,1.738*


In [7]:
# 修改行顺序，变成报告中表格的样式
results_df.iloc[[1,2,3,4,5,6,7,8,9,10,0,11],]

,re1,re2,re3,re4,re5,re6
rd_exp,0.388***,,,0.389***,,
rd_receive,,0.101***,,,0.101***,
rd_self,,,0.387***,,,0.390***
gdp_g,0.000,-0.004,0.000,-0.006,-0.009,-0.006
inf,-0.016,-0.015,-0.017,-0.018,-0.016,-0.018
pop_g,0.295***,0.254**,0.315***,0.294*,0.352**,0.286
GFCF_GDP,0.026**,0.032**,0.025**,0.002,0.015,0.002
male,,,,-0.064**,-0.035,-0.066**
age,,,,1.675*,0.779,1.738*
agesq,,,,-0.017*,-0.008,-0.017*


# 表4

In [8]:
import pandas as pd
import statsmodels.api as sm
from linearmodels.panel import PanelOLS, RandomEffects
import numpy as np

# ---------------------------------------------------
# 1. 数据准备
# ---------------------------------------------------

# 读取数据
file_path = r'2002-2017 Panel Data.xls'
data = pd.read_excel(file_path, sheet_name='Non_Manufacturing')

# 重命名列，避免使用特殊字符
data.rename(columns={'GFCF%GDP': 'GFCF_GDP'}, inplace=True)

# 选择相关列并删除缺失值，新增 'male' 和 'age'
required_columns = [
    'emp', 'rd_exp', 'gdp_g', 'inf', 'pop_g', 
    'GFCF_GDP', 'rd_receive', 'rd_self', 'Industry', 'Year',
    'male', 'age'  # 新增的变量
]
data = data[required_columns].dropna()
data = data.replace("-", np.nan)
data.loc[:,'agesq']=data.loc[:,'age']**2

# 转换数据类型
data[['rd_receive', 'rd_self', 'male', 'age']] = data[['rd_receive', 'rd_self', 'male', 'age']].apply(pd.to_numeric)

# 设置多层索引（面板数据格式：行业和年份）
data = data.set_index(['Industry', 'Year'])

# ---------------------------------------------------
# 2. 定义辅助函数
# ---------------------------------------------------

def run_model(model_type, formula, data, re_formula=None):
    """
    运行指定类型的模型。
    """
    if model_type == 'pooled':
        X = sm.add_constant(data[formula['exog']])
        y = data[formula['endog']]
        model = sm.OLS(y, X).fit()
        return model
    elif model_type == 'fe':
        model = PanelOLS.from_formula(formula['formula'], data).fit()
        return model
    elif model_type == 're':
        model = RandomEffects.from_formula(formula['formula'], data).fit()
        return model
    else:
        raise ValueError("不支持的模型类型。")

def format_pvalue(pval):
    """
    将 p 值格式化为显著性星号。
    """
    if pval < 0.01:
        return "***"
    elif pval < 0.05:
        return "**"
    elif pval < 0.1:
        return "*"
    else:
        return " "

def extract_coefficients(model, model_type):
    """
    从模型中提取系数和 p 值，并格式化显示。
    """
    coeffs = {}
    
    if model_type == 'fe':
        # 对于固定效应模型，使用实体固定效应的均值作为常数
        constant = model.estimated_effects.mean().values[0]
        coeffs['const'] = f"{constant:.3f}"  # 固定效应模型的常数没有 p 值
    else:
        # 对于其他模型，尝试提取 'const' 或 'Intercept'
        if 'const' in model.params.index:
            coeff = f"{model.params['const']:.3f}"
            star = format_pvalue(model.pvalues['const'])
            coeffs['const'] = f"{coeff}{star}"
        elif 'Intercept' in model.params.index:
            coeff = f"{model.params['Intercept']:.3f}"
            star = format_pvalue(model.pvalues['Intercept'])
            coeffs['const'] = f"{coeff}{star}"
        else:
            coeffs['const'] = None

    # 提取其他系数
    # 新增的变量包括 'male', 'age', 'rd_exp:male'（交互项）
    variables = ['rd_exp', 'rd_receive', 'rd_self', 'gdp_g', 'inf', 'pop_g', 'GFCF_GDP', 'male', 'age', 'agesq']
    for var in variables:
        if var in model.params.index:
            coeff = f"{model.params[var]:.3f}"
            pval = model.pvalues[var]
            star = format_pvalue(pval)
            coeffs[var] = f"{coeff}{star}"
        elif var.replace(':', ' * ') in model.params.index:
            # 处理交互项的命名
            interaction_var = var.replace(':', ' * ')
            if interaction_var in model.params.index:
                coeff = f"{model.params[interaction_var]:.3f}"
                pval = model.pvalues[interaction_var]
                star = format_pvalue(pval)
                coeffs[var] = f"{coeff}{star}"
            else:
                coeffs[var] = ''
        else:
            coeffs[var] = ''

    r_squared = model.rsquared
    if model_type == 're':
        r_squared = model.rsquared_between
    if model_type == 'fe':
        r_squared = model.rsquared_overall
    coeffs['R_squared'] = round(r_squared, 2) if r_squared is not None else None

    return coeffs

def prepare_results_df(models):
    """
    整理所有模型的结果到一个 DataFrame 中。
    """
    # 更新变量列表，包含新增变量
    variables = ['const', 'rd_exp', 'rd_receive', 'rd_self', 'gdp_g', 'inf', 'pop_g', 'GFCF_GDP', 'male', 'age', 'agesq']
    results = pd.DataFrame(index=variables + ['R_squared'])

    for name, model_info in models.items():
        coeffs = extract_coefficients(model_info['model'], model_info['type'])
        results[name] = [coeffs.get(var) for var in variables] + [coeffs.get('R_squared')]

    # 将 'const' 重命名为 'Constant' 以便清晰显示
    results.rename(index={'const': 'Constant'}, inplace=True)

    return results

# ---------------------------------------------------
# 3. 运行模型
# ---------------------------------------------------

# 定义每个模型的公式，所有模型均添加 'male' 和 'age'
formulas = {
    're1': {
        'type': 're',
        'formula': 'emp ~ rd_exp + gdp_g + inf + pop_g + GFCF_GDP + 1'        
    },
    're2': {
        'type': 're',
        'formula': 'emp ~ rd_receive + gdp_g + inf + pop_g + GFCF_GDP + 1'
    },
    're3': {
        'type': 're',
        'formula': 'emp ~ rd_self + gdp_g + inf + pop_g + GFCF_GDP + 1'
    },
    're4': {
        'type': 're',
        'formula': 'emp ~ rd_exp + gdp_g + inf + pop_g + GFCF_GDP + male + age + agesq + 1'
    },
    're5': {
        'type': 're',
        'formula': 'emp ~ rd_receive + gdp_g + inf + pop_g + GFCF_GDP + male + age + agesq + 1'
    },
    're6': {
        'type': 're',
        'formula': 'emp ~ rd_self + gdp_g + inf + pop_g + GFCF_GDP + male + age + agesq + 1'
    }
}

# 运行所有模型并存储在字典中
models = {}
for name, formula in formulas.items():
    models[name] = {
        'model': run_model(formula['type'], formula, data),
        'type': formula['type']
    }

# ---------------------------------------------------
# 4. 整理结果
# ---------------------------------------------------

results_df = prepare_results_df(models)

# ---------------------------------------------------
# 5. 显示结果
# ---------------------------------------------------
results_df

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/linearmodels/panel/model.py:2759: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/linearmodels/panel/model.py:2759: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/linearmodels/panel/model.py:2759: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/linearmodels/panel/model.py:2759: MissingValueWarning: 
Inputs conta

,re1,re2,re3,re4,re5,re6
Constant,1.013**,4.108***,0.703,4.399**,6.476***,-0.281
rd_exp,0.328***,,,0.362***,,
rd_receive,,-0.004,,,-0.014,
rd_self,,,0.328***,,,0.313***
gdp_g,0.010,0.004,-0.009,-0.001,-0.004,-0.008
inf,-0.025,-0.017,-0.038,-0.014,-0.012,-0.015
pop_g,0.029,0.017,0.021,0.031,0.027,0.027
GFCF_GDP,0.001**,-0.000,-0.000,0.000,-0.000,-0.000
male,,,,-0.038***,0.006,0.000
age,,,,-0.020,-0.086,0.083


In [10]:
# 修改行顺序，变成报告中表格的样式
results_df.iloc[[1,2,3,4,5,6,7,8,9,10,0,11],]

,re1,re2,re3,re4,re5,re6
rd_exp,0.328***,,,0.362***,,
rd_receive,,-0.004,,,-0.014,
rd_self,,,0.328***,,,0.313***
gdp_g,0.010,0.004,-0.009,-0.001,-0.004,-0.008
inf,-0.025,-0.017,-0.038,-0.014,-0.012,-0.015
pop_g,0.029,0.017,0.021,0.031,0.027,0.027
GFCF_GDP,0.001**,-0.000,-0.000,0.000,-0.000,-0.000
male,,,,-0.038***,0.006,0.000
age,,,,-0.020,-0.086,0.083
agesq,,,,-0.000,0.001,-0.001


# 表5

In [11]:
import pandas as pd
import statsmodels.api as sm
from linearmodels.panel import PanelOLS, RandomEffects
import numpy as np

# ---------------------------------------------------
# 1. 数据准备
# ---------------------------------------------------

# 读取数据
file_path = r'2002-2017 Panel Data.xls'
data = pd.read_excel(file_path, sheet_name='Med R')

# 重命名列，避免使用特殊字符
data.rename(columns={'GFCF%GDP': 'GFCF_GDP'}, inplace=True)

# 选择相关列并删除缺失值，新增 'male' 和 'age'
required_columns = [
    'emp', 'rd_exp', 'gdp_g', 'inf', 'pop_g', 
    'GFCF_GDP', 'rd_receive', 'rd_self', 'Industry', 'Year',
    'male', 'age'  # 新增的变量
]
data = data[required_columns].dropna()
data = data.replace("-", np.nan)
data.loc[:,'agesq']=data.loc[:,'age']**2

# 转换数据类型
data[['rd_receive', 'rd_self', 'male', 'age']] = data[['rd_receive', 'rd_self', 'male', 'age']].apply(pd.to_numeric)

# 设置多层索引（面板数据格式：行业和年份）
data = data.set_index(['Industry', 'Year'])

# ---------------------------------------------------
# 2. 定义辅助函数
# ---------------------------------------------------

def run_model(model_type, formula, data, re_formula=None):
    """
    运行指定类型的模型。
    """
    if model_type == 'pooled':
        X = sm.add_constant(data[formula['exog']])
        y = data[formula['endog']]
        model = sm.OLS(y, X).fit()
        return model
    elif model_type == 'fe':
        model = PanelOLS.from_formula(formula['formula'], data).fit()
        return model
    elif model_type == 're':
        model = RandomEffects.from_formula(formula['formula'], data).fit()
        return model
    else:
        raise ValueError("不支持的模型类型。")

def format_pvalue(pval):
    """
    将 p 值格式化为显著性星号。
    """
    if pval < 0.01:
        return "***"
    elif pval < 0.05:
        return "**"
    elif pval < 0.1:
        return "*"
    else:
        return " "

def extract_coefficients(model, model_type):
    """
    从模型中提取系数和 p 值，并格式化显示。
    """
    coeffs = {}
    
    if model_type == 'fe':
        # 对于固定效应模型，使用实体固定效应的均值作为常数
        constant = model.estimated_effects.mean().values[0]
        coeffs['const'] = f"{constant:.3f}"  # 固定效应模型的常数没有 p 值
    else:
        # 对于其他模型，尝试提取 'const' 或 'Intercept'
        if 'const' in model.params.index:
            coeff = f"{model.params['const']:.3f}"
            star = format_pvalue(model.pvalues['const'])
            coeffs['const'] = f"{coeff}{star}"
        elif 'Intercept' in model.params.index:
            coeff = f"{model.params['Intercept']:.3f}"
            star = format_pvalue(model.pvalues['Intercept'])
            coeffs['const'] = f"{coeff}{star}"
        else:
            coeffs['const'] = None

    # 提取其他系数
    # 新增的变量包括 'male', 'age', 'rd_exp:male'（交互项）
    variables = ['rd_exp', 'rd_receive', 'rd_self', 'gdp_g', 'inf', 'pop_g', 'GFCF_GDP', 'male', 'age']
    for var in variables:
        if var in model.params.index:
            coeff = f"{model.params[var]:.3f}"
            pval = model.pvalues[var]
            star = format_pvalue(pval)
            coeffs[var] = f"{coeff}{star}"
        elif var.replace(':', ' * ') in model.params.index:
            # 处理交互项的命名
            interaction_var = var.replace(':', ' * ')
            if interaction_var in model.params.index:
                coeff = f"{model.params[interaction_var]:.3f}"
                pval = model.pvalues[interaction_var]
                star = format_pvalue(pval)
                coeffs[var] = f"{coeff}{star}"
            else:
                coeffs[var] = ''
        else:
            coeffs[var] = ''

    r_squared = model.rsquared
    if model_type == 're':
        r_squared = model.rsquared_between
    if model_type == 'fe':
        r_squared = model.rsquared_overall
    coeffs['R_squared'] = round(r_squared, 2) if r_squared is not None else None

    return coeffs

def prepare_results_df(models):
    """
    整理所有模型的结果到一个 DataFrame 中。
    """
    # 更新变量列表，包含新增变量
    variables = ['const', 'rd_exp', 'rd_receive', 'rd_self', 'gdp_g', 'inf', 'pop_g', 'GFCF_GDP', 'male', 'age']
    results = pd.DataFrame(index=variables + ['R_squared'])

    for name, model_info in models.items():
        coeffs = extract_coefficients(model_info['model'], model_info['type'])
        results[name] = [coeffs.get(var) for var in variables] + [coeffs.get('R_squared')]

    # 将 'const' 重命名为 'Constant' 以便清晰显示
    results.rename(index={'const': 'Constant'}, inplace=True)

    return results

# ---------------------------------------------------
# 3. 运行模型
# ---------------------------------------------------

# 定义每个模型的公式，所有模型均添加 'male' 和 'age'
formulas = {
    're1': {
        'type': 're',
        'formula': 'emp ~ rd_exp + gdp_g + inf + pop_g + GFCF_GDP + 1'        
    },
    're2': {
        'type': 're',
        'formula': 'emp ~ rd_receive + gdp_g + inf + pop_g + GFCF_GDP + 1'
    },
    're3': {
        'type': 're',
        'formula': 'emp ~ rd_self + gdp_g + inf + pop_g + GFCF_GDP + 1'
    },
    're4': {
        'type': 're',
        'formula': 'emp ~ rd_exp + gdp_g + inf + pop_g + GFCF_GDP + male + age + 1'
    },
    're5': {
        'type': 're',
        'formula': 'emp ~ rd_receive + gdp_g + inf + pop_g + GFCF_GDP + male + age + 1'
    },
    're6': {
        'type': 're',
        'formula': 'emp ~ rd_self + gdp_g + inf + pop_g + GFCF_GDP + male + age + 1'
    }
}

# 运行所有模型并存储在字典中
models = {}
for name, formula in formulas.items():
    models[name] = {
        'model': run_model(formula['type'], formula, data),
        'type': formula['type']
    }

# ---------------------------------------------------
# 4. 整理结果
# ---------------------------------------------------

results_df = prepare_results_df(models)

# ---------------------------------------------------
# 5. 显示结果
# ---------------------------------------------------
results_df

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/linearmodels/panel/model.py:2759: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/linearmodels/panel/model.py:2759: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/linearmodels/panel/model.py:2759: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/linearmodels/panel/model.py:2759: MissingValueWarning: 
Inputs conta

,re1,re2,re3,re4,re5,re6
Constant,0.145,3.036***,-1.373**,0.287,3.076***,-1.014
rd_exp,0.268***,,,0.253***,,
rd_receive,,0.035**,,,0.032*,
rd_self,,,0.421***,,,0.406***
gdp_g,-0.002,-0.005,-0.005,-0.003,-0.006,-0.006
inf,-0.030*,-0.030,-0.034**,-0.030*,-0.031,-0.033**
pop_g,0.132,0.190,0.179,0.216,0.265,0.212
GFCF_GDP,0.060***,0.059***,0.047***,0.055***,0.054***,0.044***
male,,,,-0.004,-0.004,-0.005
age,,,,0.009,0.009,0.005


In [12]:
# 修改行顺序，变成报告中表格的样式
results_df.iloc[[1,2,3,4,5,6,7,8,9,0,10],]

,re1,re2,re3,re4,re5,re6
rd_exp,0.268***,,,0.253***,,
rd_receive,,0.035**,,,0.032*,
rd_self,,,0.421***,,,0.406***
gdp_g,-0.002,-0.005,-0.005,-0.003,-0.006,-0.006
inf,-0.030*,-0.030,-0.034**,-0.030*,-0.031,-0.033**
pop_g,0.132,0.190,0.179,0.216,0.265,0.212
GFCF_GDP,0.060***,0.059***,0.047***,0.055***,0.054***,0.044***
male,,,,-0.004,-0.004,-0.005
age,,,,0.009,0.009,0.005
Constant,0.145,3.036***,-1.373**,0.287,3.076***,-1.014


# 表6

In [13]:
import pandas as pd
import statsmodels.api as sm
from linearmodels.panel import PanelOLS, RandomEffects
import numpy as np

# ---------------------------------------------------
# 1. 数据准备
# ---------------------------------------------------

# 读取数据
file_path = r'2002-2017 Panel Data.xls'
data = pd.read_excel(file_path, sheet_name='High R')

# 重命名列，避免使用特殊字符
data.rename(columns={'GFCF%GDP': 'GFCF_GDP'}, inplace=True)

# 选择相关列并删除缺失值，新增 'male' 和 'age'
required_columns = [
    'emp', 'rd_exp', 'gdp_g', 'inf', 'pop_g', 
    'GFCF_GDP', 'rd_receive', 'rd_self', 'Industry', 'Year',
    'male', 'age','rnd_exp_x_male'  # 新增的变量
]
data = data[required_columns].dropna()
data = data.replace("-", np.nan)

# 转换数据类型
data[['rd_receive', 'rd_self', 'male', 'age' ,'rnd_exp_x_male']] = data[['rd_receive', 'rd_self', 'male', 'age','rnd_exp_x_male']].apply(pd.to_numeric)

# 设置多层索引（面板数据格式：行业和年份）
data = data.set_index(['Industry', 'Year'])

# ---------------------------------------------------
# 2. 定义辅助函数
# ---------------------------------------------------

def run_model(model_type, formula, data, re_formula=None):
    """
    运行指定类型的模型。
    """
    if model_type == 'pooled':
        X = sm.add_constant(data[formula['exog']])
        y = data[formula['endog']]
        model = sm.OLS(y, X).fit()
        return model
    elif model_type == 'fe':
        model = PanelOLS.from_formula(formula['formula'], data).fit()
        return model
    elif model_type == 're':
        model = RandomEffects.from_formula(formula['formula'], data).fit()
        return model
    else:
        raise ValueError("不支持的模型类型。")

def format_pvalue(pval):
    """
    将 p 值格式化为显著性星号。
    """
    if pval < 0.01:
        return "***"
    elif pval < 0.05:
        return "**"
    elif pval < 0.1:
        return "*"
    else:
        return " "

def extract_coefficients(model, model_type):
    """
    从模型中提取系数和 p 值，并格式化显示。
    """
    coeffs = {}
    
    if model_type == 'fe':
        # 对于固定效应模型，使用实体固定效应的均值作为常数
        constant = model.estimated_effects.mean().values[0]
        coeffs['const'] = f"{constant:.3f}"  # 固定效应模型的常数没有 p 值
    else:
        # 对于其他模型，尝试提取 'const' 或 'Intercept'
        if 'const' in model.params.index:
            coeff = f"{model.params['const']:.3f}"
            star = format_pvalue(model.pvalues['const'])
            coeffs['const'] = f"{coeff}{star}"
        elif 'Intercept' in model.params.index:
            coeff = f"{model.params['Intercept']:.3f}"
            star = format_pvalue(model.pvalues['Intercept'])
            coeffs['const'] = f"{coeff}{star}"
        else:
            coeffs['const'] = None

    # 提取其他系数
    # 新增的变量包括 'male', 'age', 'rd_exp:male'（交互项）
    variables = ['rd_exp', 'rd_receive', 'rd_self', 'gdp_g', 'inf', 'pop_g', 'GFCF_GDP', 'male', 'age', 'rnd_exp_x_male']
    for var in variables:
        if var in model.params.index:
            coeff = f"{model.params[var]:.3f}"
            pval = model.pvalues[var]
            star = format_pvalue(pval)
            coeffs[var] = f"{coeff}{star}"
        elif var.replace(':', ' * ') in model.params.index:
            # 处理交互项的命名
            interaction_var = var.replace(':', ' * ')
            if interaction_var in model.params.index:
                coeff = f"{model.params[interaction_var]:.3f}"
                pval = model.pvalues[interaction_var]
                star = format_pvalue(pval)
                coeffs[var] = f"{coeff}{star}"
            else:
                coeffs[var] = ''
        else:
            coeffs[var] = ''

    r_squared = model.rsquared
    if model_type == 're':
        r_squared = model.rsquared_between
    if model_type == 'fe':
        r_squared = model.rsquared_overall
    coeffs['R_squared'] = round(r_squared, 2) if r_squared is not None else None

    return coeffs

def prepare_results_df(models):
    """
    整理所有模型的结果到一个 DataFrame 中。
    """
    # 更新变量列表，包含新增变量
    variables = ['const', 'rd_exp', 'rd_receive', 'rd_self', 'gdp_g', 'inf', 'pop_g', 'GFCF_GDP', 'male', 'age', 'rnd_exp_x_male']
    results = pd.DataFrame(index=variables + ['R_squared'])

    for name, model_info in models.items():
        coeffs = extract_coefficients(model_info['model'], model_info['type'])
        results[name] = [coeffs.get(var) for var in variables] + [coeffs.get('R_squared')]

    # 将 'const' 重命名为 'Constant' 以便清晰显示
    results.rename(index={'const': 'Constant'}, inplace=True)

    return results

# ---------------------------------------------------
# 3. 运行模型
# ---------------------------------------------------

# 定义每个模型的公式，所有模型均添加 'male' 和 'age'
formulas = {
    're1': {
        'type': 're',
        'formula': 'emp ~ rd_exp + gdp_g + inf + pop_g + GFCF_GDP + 1'        
    },
    're2': {
        'type': 're',
        'formula': 'emp ~ rd_receive + gdp_g + inf + pop_g + GFCF_GDP + 1'
    },
    're3': {
        'type': 're',
        'formula': 'emp ~ rd_self + gdp_g + inf + pop_g + GFCF_GDP + 1'
    },
    're4': {
        'type': 're',
        'formula': 'emp ~ rd_exp + gdp_g + inf + pop_g + GFCF_GDP + male + age + 1'
    },
    're5': {
        'type': 're',
        'formula': 'emp ~ rd_receive + gdp_g + inf + pop_g + GFCF_GDP + male + age + 1'
    },
    're6': {
        'type': 're',
        'formula': 'emp ~ rd_self + gdp_g + inf + pop_g + GFCF_GDP + male + age + 1'
    },
    're7': {
        'type': 're',
        'formula': 'emp ~ rd_exp + gdp_g + inf + pop_g + GFCF_GDP + male + age + rnd_exp_x_male + 1'
    }
}

# 运行所有模型并存储在字典中
models = {}
for name, formula in formulas.items():
    models[name] = {
        'model': run_model(formula['type'], formula, data),
        'type': formula['type']
    }

# ---------------------------------------------------
# 4. 整理结果
# ---------------------------------------------------

results_df = prepare_results_df(models)

# ---------------------------------------------------
# 5. 显示结果
# ---------------------------------------------------
results_df

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/linearmodels/panel/model.py:2759: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/linearmodels/panel/model.py:2759: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/linearmodels/panel/model.py:2759: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/linearmodels/panel/model.py:2759: MissingValueWarning: 
Inputs conta

,re1,re2,re3,re4,re5,re6,re7
Constant,-0.510,3.035***,-1.158,-2.716,-0.494,-4.063*,-12.372*
rd_exp,0.377***,,,0.371***,,,1.193**
rd_receive,,0.054**,,,0.051*,,
rd_self,,,0.404***,,,0.403***,
gdp_g,0.017,0.014,0.019,0.021,0.016,0.023*,0.021
inf,-0.012,-0.008,-0.018,-0.020,-0.024,-0.029,-0.022
pop_g,0.486**,0.325,0.400*,0.418,0.517,0.349,0.377
GFCF_GDP,0.030,0.050*,0.043*,0.056*,0.074**,0.073**,0.057*
male,,,,0.032,0.036,0.039*,0.160*
age,,,,-0.012,0.009,-0.013,-0.011


In [14]:
# 修改行顺序，变成报告中表格的样式
results_df.iloc[[1,2,3,4,5,6,7,8,9,10,0,11],]

,re1,re2,re3,re4,re5,re6,re7
rd_exp,0.377***,,,0.371***,,,1.193**
rd_receive,,0.054**,,,0.051*,,
rd_self,,,0.404***,,,0.403***,
gdp_g,0.017,0.014,0.019,0.021,0.016,0.023*,0.021
inf,-0.012,-0.008,-0.018,-0.020,-0.024,-0.029,-0.022
pop_g,0.486**,0.325,0.400*,0.418,0.517,0.349,0.377
GFCF_GDP,0.030,0.050*,0.043*,0.056*,0.074**,0.073**,0.057*
male,,,,0.032,0.036,0.039*,0.160*
age,,,,-0.012,0.009,-0.013,-0.011
rnd_exp_x_male,,,,,,,-0.011
